<a href="https://colab.research.google.com/github/Mari-selvam/Voice_with_NLP/blob/main/NLP_Location_specific.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

data = pd.read_csv("/content/NLP_Sub_Location_Data - Sheet1 (1) (1).csv")
data.head()

In [ ]:
len(data)

In [ ]:
for i,j in enumerate(data['Location']):
  data.loc[i,'Location'] = str('<s>'+j+'</s>')
  # data['Location'][i] = str(j)

In [ ]:
data.head()

In [ ]:
import tensorflow as tf
from  tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
token = Tokenizer()
token.fit_on_texts(data['Location'])

In [ ]:
# token.word_index

In [ ]:
sequence = token.texts_to_sequences(data['Location'])

sequence[:10]

In [ ]:
text = pad_sequences(sequence)
text.shape

In [ ]:
len(text[0])

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y = to_categorical(data['Class'])
y

In [ ]:
locations_dict = {
    0: "APJ Block",
    1: "Main Block",
    1: "FX Block",
    2: "MBA Block",
    2: "MBA Seminar Hall",
    3: "Xerox Shop",
    4: "Exam Cell",
    4: "Controller of Examinations",
    4: "COE Block",
    5: "Basketball Ground",
    5: "Basketball Court",
    6: "Canteen",
    6: "Cafeteria",
    7: "Main Auditorium",
    8: "Mechanical Block",
    8: "Mech Block",
    9: "Library",
    10: "Playground",

}


locations_dict = {
    0: "APJ Block",
    1: "Main Block",
    2: "MBA Block",
    3: "Xerox Shop",
    4: "COE Block",
    5: "Basketball Court",
    6: "Canteen",
    7: "Main Auditorium",
    8: "Mechanical Block",
    9: "Library",
    10: "Playground",

}

In [ ]:
vocab_size = len(token.word_index) + 1
vocab_size

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense

In [ ]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000,16,input_length=text.shape[1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.Dense(11,activation='relu'),
    tf.keras.layers.Dense(11,activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(text,y,epochs=20)

In [ ]:
def preprocessing(text):

  sequence = token.texts_to_sequences(text)
  sequence = pad_sequences(sequence)
  return sequence

In [ ]:
test = preprocessing([" i want to go to the mech Block"])

In [ ]:
test

In [ ]:
output = model.predict(test)

In [ ]:
import numpy as np

In [ ]:
output = np.argmax(output)

In [ ]:
locations_dict[output]

In [ ]:
model.save("location_NLP.h5")

In [ ]:
import joblib

In [ ]:
joblib.dump(token, 'token.pkl')

In [ ]:
import tensorflow as tf
import joblib
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

model = tf.keras.models.load_model("/content/location_NLP.h5")
token = joblib.load("/content/token.pkl")

locations_dict = {
    0: "APJ Block", 1: "Main Block", 1: "FX Block", 2: "MBA Block",
    2: "MBA Seminar Hall", 3: "Xerox Shop", 4: "Exam Cell",
    4: "Controller of Examinations", 4: "COE Block", 5: "Basketball Ground",
    5: "Basketball Court", 6: "Canteen", 6: "Cafeteria", 7: "Main Auditorium",
    8: "Mechanical Block", 8: "Mech Block", 9: "Library", 10: "Playground"
}

def preprocess_text(text):
    sequences = token.texts_to_sequences([text])
    padded_sequences = pad_sequences(sequences, maxlen=model.input_shape[1])
    return padded_sequences

def main():
    text = input("Enter the location query: ")

    if not text:
        print("Error: No text provided.")
        return

    processed_text = preprocess_text(text)
    prediction = model.predict(processed_text)
    predicted_class = np.argmax(prediction)
    location = locations_dict.get(predicted_class, "Unknown Location")

    print(f"Predicted Location: {location}")

if __name__ == "__main__":
    main()


In [ ]:
import tensorflow as tf
import joblib
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

model = tf.keras.models.load_model("/content/location_NLP.h5")
token = joblib.load("/content/token.pkl")

locations_dict = {
    0: "APJ Block", 1: "Main Block", 1: "FX Block", 2: "MBA Block",
    2: "MBA Seminar Hall", 3: "Xerox Shop", 4: "Exam Cell",
    4: "Controller of Examinations", 4: "COE Block", 5: "Basketball Ground",
    5: "Basketball Court", 6: "Canteen", 6: "Cafeteria", 7: "Main Auditorium",
    8: "Mechanical Block", 8: "Mech Block", 9: "Library", 10: "Playground"
}

def print_tokenizer_vocab(tokenizer):
    print("\nTokenizer Vocabulary:")
    for word, index in tokenizer.word_index.items():
        print(f"Word: '{word}' -> Token: {index}")

def preprocess_text(text):
    sequences = token.texts_to_sequences([text])
    padded_sequences = pad_sequences(sequences, maxlen=model.input_shape[1])
    return sequences, padded_sequences

def main():
    print_tokenizer_vocab(token)

    text = input("\nEnter the location query: ")

    if not text:
        print("Error: No text provided.")
        return

    sequences, processed_text = preprocess_text(text)
    print(f"\nTokenized Sequence for '{text}': {sequences}")

    prediction = model.predict(processed_text)
    predicted_class = np.argmax(prediction)
    location = locations_dict.get(predicted_class, "Unknown Location")

    print(f"\nPredicted Location: {location}")

if __name__ == "__main__":
    main()
